In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

class jobDataset(Dataset):
	def __init__(self, mode):
		self.mode = mode

		self.job_companies=pd.read_csv('/content/drive/MyDrive/연습/채용공고추천/data/job_companies.csv')  ## (733, 3)
		self.job_tags=pd.read_csv('/content/drive/MyDrive/연습/채용공고추천/data/job_tags.csv')  ## (3477, 2)
		self.tags=pd.read_csv('/content/drive/MyDrive/연습/채용공고추천/data/tags.csv')  ## (887, 2)
		self.user_tags=pd.read_csv('/content/drive/MyDrive/연습/채용공고추천/data/user_tags.csv')  ## (17194, 2)
		self.train=pd.read_csv('/content/drive/MyDrive/연습/채용공고추천/data/train.csv')  ## (6000, 3)
		self.test=pd.read_csv('/content/drive/MyDrive/연습/채용공고추천/data/test_job.csv')  ## (2435, 2)

		self.train=self.train.merge(self.job_companies[['jobID', 'companySize']], on='jobID')
		self.test=self.test.merge(self.job_companies[['jobID', 'companySize']], on='jobID')

		self.train['companySize']=self.train['companySize'].apply(lambda x:self.midd(x) if type(x)==str else x)
		self.test['companySize']=self.test['companySize'].apply(lambda x:self.midd(x) if type(x)==str else x)
  
		train_mid = self.train['companySize'].median()
		test_mid = self.test['companySize'].median()
		self.train['companySize']=self.train['companySize'].fillna(train_mid)
		self.test['companySize']=self.test['companySize'].fillna(test_mid)


		self.train['tag_match_rate']=self.tag_match(self.train)
		self.test['tag_match_rate']=self.tag_match(self.test)

		'''
		le = LabelEncoder()  
		self.train['jobID'] = le.fit_transform(self.train['jobID'].values.reshape(-1, 1))
		for label in np.unique(self.test['jobID']):
			if label not in le.classes_: le.classes_=np.append(le.classes_,label)
		self.test['jobID']=le.transform(self.test['jobID'].values.reshape(-1, 1))
		'''
		oe=OneHotEncoder(handle_unknown='ignore')
		oe_result = oe.fit_transform(self.train['jobID'].values.reshape(-1, 1)).toarray()
		sub=pd.DataFrame(data=oe_result, columns=oe.get_feature_names(['jobID']))
		self.train=pd.concat([self.train, sub], axis=1)

		oe_result = oe.transform(self.test['jobID'].values.reshape(-1, 1)).toarray()
		sub=pd.DataFrame(data=oe_result, columns=oe.get_feature_names(['jobID']))
		self.test=pd.concat([self.test, sub], axis=1)
		

		le2 = LabelEncoder()  
		self.train['userID'] = le2.fit_transform(self.train['userID'].values.reshape(-1, 1))
		self.test['userID']=le2.transform(self.test['userID'].values.reshape(-1, 1))


		self.train['companySize']=self.train.companySize.apply(lambda x: np.log(x, where =(x!=0)))
		self.train['tag_match_rate']=self.train.tag_match_rate.apply(lambda x: np.log(x) if x!=0 else x)

		self.test['companySize']=self.test.companySize.apply(lambda x: np.log(x, where =(x!=0)))
		self.test['tag_match_rate']=self.test.tag_match_rate.apply(lambda x: np.log(x) if x!=0 else x)
  

		st_scaler1 = StandardScaler()
		self.train['companySize']=st_scaler1.fit_transform(self.train['companySize'].values.reshape(-1,1))
		self.test['companySize']=st_scaler1.transform(self.test['companySize'].values.reshape(-1,1))

		st_scaler2 = StandardScaler()
		self.train['tag_match_rate']=st_scaler2.fit_transform(self.train['tag_match_rate'].values.reshape(-1,1))
		self.test['tag_match_rate']=st_scaler2.transform(self.test['tag_match_rate'].values.reshape(-1,1))

		if self.mode=='Train':
			self.label = self.train['applied']
			self.data = self.train.drop('applied', axis=1)
			self.label = torch.FloatTensor(self.label.values)
		else: self.data=self.test
		self.data=self.data.drop('jobID',axis=1)  #################  one hot endcoding 시에 추가
		self.data.head(5)
		self.data = torch.FloatTensor(self.data.values)

		print(self.data.shape)
  

	def __len__(self):
		return self.data.shape[0]
  
	def __getitem__(self, idx):
		if self.mode=="Train": 
			return self.data[idx], self.label[idx]
		else: return self.data[idx]


	def midd(self, data):
		if data=='1000 이상': return 1000
		data=data.split('-')
		return (int(data[0])+int(data[1]))//2


	def tag_match(self, data):
		tag = []
		user=data['userID']
		job=data['jobID']
		for u, j in zip(user, job):
			# user_tags에서 현재 user와 일치하는 userID의 tagID만 가져옵니다.user의 관심사 키워드를 가져옵니다.
			user_tag=self.user_tags[self.user_tags['userID']==u]['tagID'].values

			# job_tags에서 현재 user의 job과 일치하는 jobID의 tagID만 가져옵니다. job의 키워드를 가져옵니다.
			job_tag=self.job_tags[self.job_tags['jobID']==j]['tagID'].values

			job_tag_total=len(job_tag) # job의 해당 키워드가 총 몇개인지
			cnt=0
			for i in user_tag:
				# user의 관심 키워드가 job의 키워드와 일치한다면 cnt를 1증가시켜 줍니다.
				if i in job_tag: cnt+=1 
		
			# job의 전체 키워드중 몇개가 user의 관심사 키워드와 일치했는지 비율을 계산해줍니다.
			tag.append(cnt/job_tag_total)
		return tag
jobDataset('Train')

torch.Size([6000, 3])


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please chan

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
